In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [2]:
df = pd.read_csv('Data/Order_Agg.csv').sort_values("AccountId") 

In [3]:
df_cleaned = df

In [4]:
#valid_groups = df.groupby("AccountId")["HasOrder"].sum()
#valid_accounts = valid_groups[valid_groups > 0].index
#df_new = df[df["AccountId"].isin(valid_accounts)]
#df = df_new

In [5]:
#Normaliza a coluna entre 0 e 1
#for column in [c for c in df_cleaned.columns if c not in ['AccountId', 'PorductId', 'HasOrder']]:
#    df_cleaned[column] = (df_cleaned[column] - df_cleaned[column].min()) / (df_cleaned[column].max() - df_cleaned[column].min())

In [6]:
X = df_cleaned[
    [
        'TotalPrice',
        'AvgPrice',
        'TotalQuantity',
        'AvgQuantity',
        'Recency',
        'AvgRecency',
        'ProductPopularity',
        'TotalPriceProportion',
        'TotalQuantityProportion',
        'OrderCount',
        'Frequency',
        'RFM'
    ]
]
y = df_cleaned["HasOrder"] == 1
#X = df_cleaned[["TotalPrice", "AvgPrice", "TotalQuantity", "AvgQuantity"]]
#y = df_cleaned["HasOrder"]
groups = df.groupby("AccountId").size().to_list()  # tamanho do grupo por usuário

train_data = lgb.Dataset(X, label=y, group=groups)

params = {
    "objective": "lambdarank",   # Ranking
    "metric": "ndcg",            # Avaliação por ranking
    "ndcg_eval_at": [5, 10],         # Avaliação no top-5
    "learning_rate": 0.01,
    "num_leaves": 62,
    "min_data_in_leaf": 30
}

print(y.max())

model = lgb.train(params, train_data, num_boost_round=500)

True
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.289642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2522
[LightGBM] [Info] Number of data points in the train set: 3613231, number of used features: 12
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [13]:
# Simulação de features para um novo usuário com 10 produtos candidatos
X_new = df_cleaned[df_cleaned['AccountId'] == '0018c00002SwxD0AAJ'][[
        'TotalPrice',
        'AvgPrice',
        'TotalQuantity',
        'AvgQuantity',
        'Recency',
        'AvgRecency',
        'ProductPopularity',
        'TotalPriceProportion',
        'TotalQuantityProportion',
        'OrderCount',
        'Frequency',
        'RFM'
    ]]
product_ids = df_cleaned[df_cleaned['AccountId'] == '0018c00002SwxD0AAJ']['ProductCode'].to_list() #list(range(new_products))

# Predição
scores = model.predict(X_new)

# Ordenando por relevância
ranking = sorted(zip(product_ids, scores), key=lambda x: x[1], reverse=True)

print("Ranking de produtos recomendados:")
for prod, score in ranking:
    print(f"Produto {prod} - score {score:.4f}")

Ranking de produtos recomendados:
Produto 77303543.0 - score 9.3501
Produto 77200654.0 - score 9.3501
Produto 40370020.0 - score 9.3501
Produto 77218754.0 - score 9.3501
Produto 77459534.0 - score 9.3501
Produto 77600615.0 - score 9.3501
Produto 77855764.0 - score 9.3501
Produto 77395085.0 - score 9.3501
Produto 78502750.0 - score 9.3501
Produto 40370018.0 - score 9.3501
Produto 77819051.0 - score 9.3501
Produto 77559524.0 - score 9.3501
Produto 78506750.0 - score 9.3501
Produto 77704432.0 - score 9.3501
Produto 44452209.0 - score -9.3501
Produto 44660113.0 - score -9.3501
Produto 43610022.0 - score -9.3501
Produto 44660120.0 - score -9.3501
Produto 44660117.0 - score -9.3501
Produto 44660116.0 - score -9.3501
Produto 44869107.0 - score -9.3501
Produto 44660114.0 - score -9.3501
Produto 44810107.0 - score -9.3501
Produto 44660123.0 - score -9.3501
Produto 44869109.0 - score -9.3501
Produto 44660121.0 - score -9.3501
Produto 44660118.0 - score -9.3501
Produto 44811107.0 - score -9.3501
